In [1]:
# https://graphacademy.neo4j.com/courses/app-python/2-interacting/2-results/

from neo4j import GraphDatabase
import os

# Set up a neo4j driver instance
uri = "neo4j://35.226.169.213:7687"
username = "neo4j"
password = "neo4jtest"
driver = GraphDatabase.driver(uri, auth=(username, password))

In [19]:
# Get Actors by Movie title

# Unit of work
def get_actors(tx, movie): # (1)
    result = tx.run("""
        MATCH (p:Person)-[:ACTED_IN]->(:Movie {title: $title})
        RETURN p
    """, title=movie)

    # Peeking at Result
    peek = result.peek()
    print(peek)

    # Get Keys
    print("test0: ", result.keys())
    # result_test = result

    # for record_test in result_test:
    #     print("test:" ,record_test["p"])

    # Access the `p` value from each record
    return [ record["p"] for record in result ]

<Record p=<Node element_id='4:df5bea23-5ef6-42e6-8ad8-95e3bc16d3b8:222' labels=frozenset({'Person'}) properties={'born': 1959, 'name': 'Patricia Clarkson'}>>
['p']
test2 : [<Node element_id='4:df5bea23-5ef6-42e6-8ad8-95e3bc16d3b8:222' labels=frozenset({'Person'}) properties={'born': 1959, 'name': 'Patricia Clarkson'}>, <Node element_id='4:df5bea23-5ef6-42e6-8ad8-95e3bc16d3b8:220' labels=frozenset({'Person'}) properties={'born': 1968, 'name': 'Sam Rockwell'}>, <Node element_id='4:df5bea23-5ef6-42e6-8ad8-95e3bc16d3b8:218' labels=frozenset({'Person'}) properties={'born': 1957, 'name': 'Michael Clarke Duncan'}>, <Node element_id='4:df5bea23-5ef6-42e6-8ad8-95e3bc16d3b8:221' labels=frozenset({'Person'}) properties={'born': 1955, 'name': 'Gary Sinise'}>, <Node element_id='4:df5bea23-5ef6-42e6-8ad8-95e3bc16d3b8:219' labels=frozenset({'Person'}) properties={'born': 1953, 'name': 'David Morse'}>]
test3: <Node element_id='4:df5bea23-5ef6-42e6-8ad8-95e3bc16d3b8:222' labels=frozenset({'Person'}) pr

In [24]:
# Extract a value
def get_actors_value(tx, movie):
    result = tx.run("""
        MATCH (p:Person)-[r:ACTED_IN]->(m:Movie {title: $title})
        RETURN p.name AS name, m.title AS title, r.roles AS roles
    """, title=movie)

    return result.value("name", False)
    # Returns the `name` value, or False if unavailable


In [39]:
def get_actors_values(tx, movie):
    result = tx.run("""
        MATCH (p:Person)-[r:ACTED_IN]->(m:Movie {title: $title})
        RETURN p.name AS name, m.title AS title, r.roles AS roles
    """, title=movie)

    print("test0:", result.keys())

    return result.values("name", "title", "roles")

In [41]:
# Single Result  
# If you only expect a single record, you can use the single() method on the result to return the first record.
def get_actors_single(tx, movie):
    result = tx.run("""
        MATCH (p:Person)-[:ACTED_IN]->(:Movie {title: $title})
        RETURN p
    """, title=movie)

    return result.single()

In [42]:
# Open a Session
with driver.session() as session:
    # Run the unit of work within a Read Transaction
    # actors = session.execute_read(get_actors, movie="The Green Mile") # Get Actors    ==> 잘 이해가 안된다
    # actors = session.execute_read(get_actors_value, movie="The Green Mile") # Get value
    # actors = session.execute_read(get_actors_values, movie="The Green Mile") # Get values
    actors = session.execute_read(get_actors_single, movie="The Green Mile") # Get single
    
    print("test2 :",  actors)

    for record in actors:
        print("test3:", record)
        # print("test4:", record["p"])    # Get Actors

    session.close()

test2 : <Record p=<Node element_id='4:df5bea23-5ef6-42e6-8ad8-95e3bc16d3b8:222' labels=frozenset({'Person'}) properties={'born': 1959, 'name': 'Patricia Clarkson'}>>
test3: <Node element_id='4:df5bea23-5ef6-42e6-8ad8-95e3bc16d3b8:222' labels=frozenset({'Person'}) properties={'born': 1959, 'name': 'Patricia Clarkson'}>


c:\Users\haid\anaconda3\lib\site-packages\neo4j\_sync\work\result.py:465: UserWarning: Expected a result with a single record, but found multiple.
  warn("Expected a result with a single record, "


In [45]:
## consume 은 사용 방법을 몰라서 Test 를 해 보지 못했다.

def get_actors_consume(tx, name):
    result = tx.run("""
        MERGE (p:Person {name: $name})
        RETURN p
    """, name=name)

    info = result.consume()

    # The time it took for the server to have the result available. (milliseconds)
    print("test_info1: ", info.result_available_after)

    # The time it took for the server to consume the result. (milliseconds)
    print("test_info2: ", info.result_consumed_after)

def create_person(name):
    # Create a Session for the `neo4j` database
    session = driver.session(database="neo4j")

    # Create a node within a write transaction
    record = session.execute_write(get_actors_consume, name=name)

    # Get the `p` value from the first record
    person = record["p"]
    print(person)

    # Close the session
    session.close()



In [46]:
create_person("KM Kang")

test_info1:  16
test_info2:  5


TypeError: 'NoneType' object is not subscriptable